In [1]:
import os, json, math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# -------------------- Meine empfohlenen Defaults --------------------
ROUNDTRIP_BPS_DEFAULT = 15.0          # ← Hauptszenario: 15 bps Roundtrip (AAPL, H≈5)
SENSI_BPS = [5.0, 10.0, 15.0, 25.0, 50.0, 100.0]   # ← Sensitivität
SLIPPAGE_BPS_PER_LEG = 2.0            # ← 2 bps je Leg (Entry/Exit) ≈ +4 bps RT effektiv

In [3]:
# -------------------- Artefakte laden --------------------
ROOT = Path("..")
with open(ROOT/"config.json","r") as f:
    C = json.load(f)

RESULTS_DIR = Path(C.get("results_dir","../results"))
runs = sorted(RESULTS_DIR.glob("*_lstm"), key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, "Kein *_lstm Run-Ordner gefunden."
RUN_DIR = runs[0]

with open(RUN_DIR/"config.json","r") as f:
    RCFG = json.load(f)

TRAIN_CSV = Path(RCFG["train_csv"])
H = int(RCFG["horizon"])
LOOKBACK = int(RCFG["lookback"])

preds = pd.read_csv(RUN_DIR/"preds_test.csv", parse_dates=["timestamp"]).set_index("timestamp").sort_index()
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()

close = df["close"].reindex(preds.index)
assert close.notna().all(), "Close-Preise fehlen auf Test-Index."

with open(RUN_DIR/"evaluation.json","r") as f:
    EVAL = json.load(f)
thr = float(EVAL["threshold_selection"]["threshold"])

proba_used = preds["y_proba_used"].values
sig_t  = (proba_used >= thr).astype(int)
sig_t1 = pd.Series(sig_t, index=preds.index).shift(1).fillna(0).astype(int).values

fwd_logret = (np.log(close.shift(-H)) - np.log(close)).values

In [4]:
# -------------------- Kostenmodell --------------------
def log_cost_for_roundtrip(rt_bps: float, slip_bps_per_leg: float = 0.0) -> float:
    """
    Exakte Log-Kosten: je Leg c_leg = (rt_bps/2 + slippage_per_leg) / 1e4
    Roundtrip-Logkosten = ln(1-c_leg) + ln(1-c_leg) = 2*ln(1-c_leg)
    """
    per_leg_frac = (rt_bps/2.0 + slip_bps_per_leg) / 1e4
    per_leg_frac = max(0.0, min(0.99, per_leg_frac))
    return 2.0 * np.log(1.0 - per_leg_frac)

def equity_from_signals(sig, logrets, rt_bps, slip_per_leg_bps=0.0):
    sig = np.asarray(sig, int)
    gross = (sig * np.nan_to_num(logrets, nan=0.0))
    trades = sig.astype(bool)
    log_cost = log_cost_for_roundtrip(rt_bps, slip_per_leg_bps)
    net = gross + trades.astype(float) * log_cost
    eq = np.exp(pd.Series(net).cumsum())
    return eq, gross, net, trades.sum()

def stats_from_equity(eq: pd.Series, net_logrets: pd.Series, periods_per_year=252):
    eq = pd.Series(eq).dropna()
    T = len(eq) / periods_per_year if len(eq) else 0.0
    def cagr():
        return float((eq.iloc[-1] / eq.iloc[0])**(1.0/max(T,1e-12)) - 1.0)
    def sharpe():
        lr = pd.Series(net_logrets).dropna()
        mu = lr.mean() * periods_per_year
        sd = lr.std(ddof=1) * math.sqrt(periods_per_year)
        return float(mu / (sd + 1e-12)) if len(lr) > 1 else 0.0
    def max_dd():
        cum = np.log(eq.values)
        peak = np.maximum.accumulate(cum)
        dd = np.exp(cum - peak) - 1.0
        return float(dd.min())
    return dict(CAGR=cagr(), Sharpe=sharpe(), MaxDD=max_dd(), final_equity=float(eq.iloc[-1]))

# Buy&Hold auf dem gleichen Fenster
bh_log = (np.log(close) - np.log(close.iloc[0])).fillna(0.0)
bh_eq = np.exp(bh_log)

In [5]:
# -------------------- Hauptszenario --------------------
main_rt = ROUNDTRIP_BPS_DEFAULT
eq_t,  g_t,  n_t,  n_tr_t  = equity_from_signals(sig_t,  fwd_logret, main_rt, SLIPPAGE_BPS_PER_LEG)
eq_t1, g_t1, n_t1, n_tr_t1 = equity_from_signals(sig_t1, fwd_logret, main_rt, SLIPPAGE_BPS_PER_LEG)

stats_t  = stats_from_equity(eq_t,  n_t)
stats_t1 = stats_from_equity(eq_t1, n_t1)

print(f"[Block 6] Hauptszenario RT={main_rt:.0f} bps | Slippage/Leg={SLIPPAGE_BPS_PER_LEG:.1f} bps")
print("Entry@t : trades=", n_tr_t,  "|", stats_t)
print("Entry@t+1: trades=", n_tr_t1, "|", stats_t1)

# Plot speichern
figdir = RUN_DIR/"figures"
figdir.mkdir(parents=True, exist_ok=True)
plt.figure(figsize=(8,4))
plt.plot(preds.index, eq_t,  label=f"Entry@t (net, {main_rt:.0f}bps, slip {SLIPPAGE_BPS_PER_LEG:.0f}/leg)")
plt.plot(preds.index, eq_t1, label=f"Entry@t+1 (net, {main_rt:.0f}bps, slip {SLIPPAGE_BPS_PER_LEG:.0f}/leg)")
plt.plot(preds.index, bh_eq.reindex(preds.index), label="Buy & Hold", linestyle="--")
plt.title(f"Equity mit Kosten (H={H})")
plt.legend(); plt.tight_layout()
plt.savefig(figdir/"equity_costed.png", dpi=160); plt.close()

[Block 6] Hauptszenario RT=15 bps | Slippage/Leg=2.0 bps
Entry@t : trades= 192 | {'CAGR': 0.04458432743436713, 'Sharpe': 0.08722507342850089, 'MaxDD': -0.6953498570518268, 'final_equity': 1.081379254707068}
Entry@t+1: trades= 191 | {'CAGR': -0.012312332789380487, 'Sharpe': -0.024303887046502683, 'MaxDD': -0.6809545725607603, 'final_equity': 0.9780239653946093}


In [6]:
# -------------------- Sensitivität --------------------
rows = []
for rt in SENSI_BPS:
    eqA, gA, nA, ntrA = equity_from_signals(sig_t,  fwd_logret, rt, SLIPPAGE_BPS_PER_LEG)
    eqB, gB, nB, ntrB = equity_from_signals(sig_t1, fwd_logret, rt, SLIPPAGE_BPS_PER_LEG)
    rows.append(dict(model="Entry@t",   roundtrip_bps=rt, trades=int(ntrA), **stats_from_equity(eqA, nA)))
    rows.append(dict(model="Entry@t+1", roundtrip_bps=rt, trades=int(ntrB), **stats_from_equity(eqB, nB)))

sensi = pd.DataFrame(rows).sort_values(["model","roundtrip_bps"])
sensi_path = RUN_DIR/"cost_sensitivity.csv"
sensi.to_csv(sensi_path, index=False)

print("\nSensitivität (bps):")
print(sensi.to_string(index=False, float_format=lambda x: f"{x:,.4f}"))


Sensitivität (bps):
    model  roundtrip_bps  trades    CAGR  Sharpe   MaxDD  final_equity
  Entry@t         5.0000     192  0.1627  0.3014 -0.6829        1.3105
  Entry@t        10.0000     192  0.1021  0.1943 -0.6892        1.1904
  Entry@t        15.0000     192  0.0446  0.0872 -0.6953        1.0814
  Entry@t        25.0000     192 -0.0616 -0.1271 -0.7073        0.8923
  Entry@t        50.0000     192 -0.2823 -0.6626 -0.7352        0.5516
  Entry@t       100.0000     192 -0.5807 -1.7239 -0.8322        0.2104
Entry@t+1         5.0000     191  0.0987  0.1847 -0.6679        1.1840
Entry@t+1        10.0000     191  0.0417  0.0802 -0.6745        1.0761
Entry@t+1        15.0000     191 -0.0123 -0.0243 -0.6810        0.9780
Entry@t+1        25.0000     191 -0.1122 -0.2334 -0.6935        0.8078
Entry@t+1        50.0000     191 -0.3201 -0.7556 -0.7227        0.5006
Entry@t+1       100.0000     191 -0.6016 -1.7897 -0.8173        0.1919


In [7]:
# -------------------- evaluation.json anreichern --------------------
cost_block = {
    "roundtrip_bps_default": main_rt,
    "slippage_bps_per_leg": SLIPPAGE_BPS_PER_LEG,
    "entry_t":  {"trades": int(n_tr_t),  **stats_t},
    "entry_t1": {"trades": int(n_tr_t1), **stats_t1},
    "sensitivity_csv": str(sensi_path.as_posix()),
    "equity_costed_png": str((figdir/"equity_costed.png").as_posix())
}
EVAL["backtest_costs"] = cost_block
with open(RUN_DIR/"evaluation.json","w") as f:
    json.dump(EVAL, f, indent=2)

print("\nBlock 6 abgeschlossen →")
print(" - figures/equity_costed.png")
print(" - cost_sensitivity.csv")
print(" - evaluation.json (mit backtest_costs)")


Block 6 abgeschlossen →
 - figures/equity_costed.png
 - cost_sensitivity.csv
 - evaluation.json (mit backtest_costs)
